<a href="https://colab.research.google.com/github/okolenchuk/DS_HW_RNN/blob/main/Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Обучите простую рекуррентную нейронную сеть (без GRU/LSTM, без внимания) решать задачу дешифровки шифра Цезаря:
1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на N каждой буквы). Например если N=2, то буква A переходит в букву C. Можно поиграться с
языком на выбор (немецкий, русский и т.д.)
2. Создать архитектуру рекуррентной нейронной сети.
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза).
4. Проверить качество модели.


2 балла за правильно выполненное задание.

In [38]:
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings
from random import shuffle
import time
import math
import numpy

warnings.filterwarnings("ignore")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
numpy.set_printoptions(threshold=1000)

Загружаем файл с текстом из интернета

In [39]:
!wget https://tululu.org/txt.php?id=51554

--2022-09-17 22:44:29--  https://tululu.org/txt.php?id=51554
Resolving tululu.org (tululu.org)... 104.21.82.5, 172.67.167.88, 2606:4700:3034::6815:5205, ...
Connecting to tululu.org (tululu.org)|104.21.82.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 667704 (652K) [text/plain]
Saving to: ‘txt.php?id=51554.2’

txt.php?id=51554.2  100%[===================>] 652.05K  4.23MB/s    in 0.2s    

2022-09-17 22:44:29 (4.23 MB/s) - ‘txt.php?id=51554.2’ saved [667704/667704]



In [40]:
file_path = '/content/txt.php?id=51554'
string_size = 60
batch_size = 10
NUM_EPOCHS = 20
LEARNING_RATE = 0.01

Класс для кодирования текста по правилам шифта Цезаря с заданным шагом, и раскодировки для проверки на незнакомом корпусе текста.

In [41]:
class Cesar(object):
    def __init__(self, step):
        self.step = step
        self.alphabet = ''
        self.len_alphabet = 0

    def alphabet_from_file(self, file_path):
        with open(file_path) as file:
            while True:
                text = file.read(string_size)
                if not text:
                    break
                for ch in text:
                    if ch not in self.alphabet:
                        self.alphabet += ch
        self.alphabet = re.sub(r'[^a-zA-Z.!? ]+', r'', ''.join(sorted(self.alphabet)))
        self.len_alphabet = len(self.alphabet)

    def encode(self, text):
        res = ''
        for c in re.sub(r'[^a-zA-Z.!? ]+', r' ', text):
            if c in self.alphabet:
                res += self.alphabet[(self.alphabet.index(c) + self.step) % len(self.alphabet)]
        return res

    def decode(self, text):
        res = ''
        for c in text:
            res += self.alphabet[(self.alphabet.index(c) - self.step% len(self.alphabet))]
        return res

coder = Cesar(2)
coder.alphabet_from_file(file_path)
alpha = coder.alphabet
alpha

' !.?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

In [42]:
a = coder.encode(sentence)
b = coder.decode(a)


print(sentence)
print(a,b, sep='\n')
print(alpha[alpha.index(' ') + coder.step% len(alpha)])

Jupyter Notebook
Jupyter notebook, formerly known as the IPython notebook, is a flexible tool that helps you create readable analyses, as you can keep code, images, comments, formulae and plots together. In this post, we’ve collected some of the top Jupyter notebook tips to quickly turn you into a Jupyter power user!

(This post is based on a post that originally appeared on Alex Rogozhnikov’s blog, ‘Brilliantly Wrong’. We have expanded the post and will continue to do so over time — if you have a suggestion please let us know. Thanks to Alex for graciously letting us republish his work here.)

Jupyter is quite extensible, supports many programming languages and is easily hosted on your computer or on almost any server — you only need to have ssh or http access. Best of all, it’s completely free. Now let’s dive in to our list of 28 (and counting!) Jupyter notebook tips!
Lwr vgt.Pqvgdqqm.Lwr vgt.pqvgdqqm..hqtogtn .mpqyp.cu.vjg.KR vjqp.pqvgdqqm..ku.c.hngzkdng.vqqn.vjcv.jgnru. qw.etgcvg.t

In [71]:
def make_dataset(file_path, step):
  text_array = []
  with open(file_path) as file:
    while True:
      text = file.read(string_size)
      if not text:
        break
      text_array.append(re.sub(r'[^a-zA-Z.!? ]', r' ', text))
  del text_array[-1]  
  y_train = torch.tensor([[alpha.find(y) for y in lines] for lines in text_array[:len(text_array)//5]])
  x_train = torch.tensor([[alpha.find(x) for x in coder.encode(lines)] for lines in text_array[:len(text_array)//5]])

  y_test = torch.tensor([[alpha.find(y) for y in lines] for lines in text_array[len(text_array)//5:]])
  x_test = torch.tensor([[alpha.find(x) for x in coder.encode(lines)] for lines in text_array[len(text_array)//5:]])

  return x_train, y_train, x_test, y_test

In [72]:
x_train, y_train, x_test, y_test = make_dataset(file_path, string_size)

In [73]:
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, x, y):
        super().__init__()
        self._len = len(x)
        self.y = y.to(DEVICE)
        self.x = x.to(DEVICE)
    
    def __len__(self):
        return self._len

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [74]:
train_ds = torch.utils.data.DataLoader(MyDataset(x_train, y_train), 
                                       batch_size=batch_size, 
                                       shuffle=True)
test_ds = torch.utils.data.DataLoader(MyDataset(x_test, y_test), 
                                       batch_size=batch_size, 
                                       shuffle=True)

In [118]:
class RNNModel(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(60, 32)
        self.rnn = torch.nn.RNN(32, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(alpha))

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        o, h = self.rnn(embed)
        return self.linear(o)

In [119]:
model = RNNModel().to(DEVICE)
loss = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [120]:
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    for x_in, y_in in train_ds:
        x_in = x_in.to(DEVICE)
        y_in = y_in.view(1, -1).squeeze().to(DEVICE)
        optimizer.zero_grad()
        out = model.forward(x_in).view(-1, len(alpha))
        l = loss(out, y_in)
        train_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        train_acc += batch_acc.sum().item() / batch_acc.shape[0]
        l.backward()
        optimizer.step()
        iter_num += 1
    print(
        f"Epoch: {epoch}, loss: {train_loss:.4f}, acc: "
        f"{train_acc / iter_num:.4f}",
        end=" | "
    )
    test_loss, test_acc, iter_num = .0, .0, .0
    model.eval()
    for x_in, y_in in test_ds:
        x_in = x_in.to(DEVICE)
        y_in = y_in.view(1, -1).squeeze()
        out = model.forward(x_in).view(-1, len(alpha)).to(DEVICE)
        l = loss(out, y_in)
        test_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        test_acc += batch_acc.sum().item() / batch_acc.shape[0]
        iter_num += 1
    print(
        f"test loss: {test_loss:.4f}, test acc: {test_acc / iter_num:.4f} | "
        f"{time.time() - start_epoch_time:.2f} sec."
    )

Epoch: 0, loss: 565.0888, acc: 0.6169 | test loss: 1475.6973, test acc: 0.7613 | 6.01 sec.
Epoch: 1, loss: 275.8578, acc: 0.8062 | test loss: 818.9610, test acc: 0.8682 | 6.06 sec.
Epoch: 2, loss: 165.7900, acc: 0.9142 | test loss: 533.7789, test acc: 0.9401 | 5.97 sec.
Epoch: 3, loss: 112.5957, acc: 0.9493 | test loss: 379.8665, test acc: 0.9559 | 5.89 sec.
Epoch: 4, loss: 82.4287, acc: 0.9637 | test loss: 289.4384, test acc: 0.9675 | 5.86 sec.
Epoch: 5, loss: 64.2933, acc: 0.9699 | test loss: 233.6759, test acc: 0.9690 | 6.00 sec.
Epoch: 6, loss: 52.7821, acc: 0.9710 | test loss: 197.0739, test acc: 0.9700 | 6.72 sec.
Epoch: 7, loss: 45.0216, acc: 0.9726 | test loss: 171.5808, test acc: 0.9717 | 5.91 sec.
Epoch: 8, loss: 39.4975, acc: 0.9752 | test loss: 152.9724, test acc: 0.9756 | 5.90 sec.
Epoch: 9, loss: 35.3955, acc: 0.9786 | test loss: 138.8154, test acc: 0.9779 | 5.84 sec.
Epoch: 10, loss: 32.2291, acc: 0.9792 | test loss: 127.6863, test acc: 0.9779 | 5.91 sec.
Epoch: 11, loss

In [121]:
sentence = """Jupyter Notebook
Jupyter notebook, formerly known as the IPython notebook, is a flexible tool that helps you create readable analyses, as you can keep code, images, comments, formulae and plots together. In this post, we’ve collected some of the top Jupyter notebook tips to quickly turn you into a Jupyter power user!

(This post is based on a post that originally appeared on Alex Rogozhnikov’s blog, ‘Brilliantly Wrong’. We have expanded the post and will continue to do so over time — if you have a suggestion please let us know. Thanks to Alex for graciously letting us republish his work here.)

Jupyter is quite extensible, supports many programming languages and is easily hosted on your computer or on almost any server — you only need to have ssh or http access. Best of all, it’s completely free. Now let’s dive in to our list of 28 (and counting!) Jupyter notebook tips!"""
encrypted_sentence = coder.encode(sentence)
encrypted_sentence_idx = [alpha.find(i) for i in encrypted_sentence]
result = model(torch.tensor([encrypted_sentence_idx]).to(DEVICE)).argmax(dim=2)
deencrypted_sentence = "".join([alpha[i.item()] for i in result.flatten()])
print(f"Encrypted sentence is : {encrypted_sentence}")
print("-" * 20)
print(deencrypted_sentence)
print(coder.decode(encrypted_sentence))

Encrypted sentence is : Lwr vgt.Pqvgdqqm.Lwr vgt.pqvgdqqm..hqtogtn .mpqyp.cu.vjg.KR vjqp.pqvgdqqm..ku.c.hngzkdng.vqqn.vjcv.jgnru. qw.etgcvg.tgcfcdng.cpcn ugu..cu. qw.ecp.mggr.eqfg..kocigu..eqoogpvu..hqtowncg.cpf.rnqvu.vqigvjgtA.Kp.vjku.rquv..yg.xg.eqnngevgf.uqog.qh.vjg.vqr.Lwr vgt.pqvgdqqm.vkru.vq.swkemn .vwtp. qw.kpvq.c.Lwr vgt.rqygt.wugt?.Vjku.rquv.ku.dcugf.qp.c.rquv.vjcv.qtkikpcnn .crrgctgf.qp.Cngz.Tqiq!jpkmqx.u.dnqi...Dtknnkcpvn .Ytqpi.A.Yg.jcxg.gzrcpfgf.vjg.rquv.cpf.yknn.eqpvkpwg.vq.fq.uq.qxgt.vkog...kh. qw.jcxg.c.uwiiguvkqp.rngcug.ngv.wu.mpqyA.Vjcpmu.vq.Cngz.hqt.itcekqwun .ngvvkpi.wu.tgrwdnkuj.jku.yqtm.jgtgA.Lwr vgt.ku.swkvg.gzvgpukdng..uwrrqtvu.ocp .rtqitcookpi.ncpiwcigu.cpf.ku.gcukn .jquvgf.qp. qwt.eqorwvgt.qt.qp.cnoquv.cp .ugtxgt... qw.qpn .pggf.vq.jcxg.uuj.qt.jvvr.ceeguuA.Dguv.qh.cnn..kv.u.eqorngvgn .htggA.Pqy.ngv.u.fkxg.kp.vq.qwt.nkuv.qh....cpf.eqwpvkpi?..Lwr vgt.pqvgdqqm.vkru?
--------------------
 upyter ootebook  upyter notebook  formerly known as the Itython notebook  is